### Connect to Gremlin Server

In [5]:
import os
import sys
from pathlib import Path

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)
    
from graph.base import g

from dotenv import load_dotenv

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager
from queue import Queue

# Queue to hold statements
statement_queue = Queue()

# Function to add statement to queue
def add_to_queue(statement):
    statement_queue.put(statement)

@contextmanager
def execute_gremlin_statements():
    try:
        yield add_to_queue
    finally:
        statements = []
        while not statement_queue.empty():
            statements.append(statement_queue.get())
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(statement) for statement in statements]
            results = [future.result() for future in futures]

        for result in results:
            print(result)


RuntimeError: Cannot run the event loop while another loop is running

### Clear Database

In [2]:
with execute_gremlin_statements() as add_statement:
    add_statement(lambda: g.V().drop().iterate())

[['V'], ['drop'], ['none']]


### Add some Test Data

In [3]:
from gremlin_python import statics
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *


with execute_gremlin_statements() as add_statement:
    # Add vertices
    add_statement(lambda: g.addV('person').property('name', 'Bob').next())
    add_statement(lambda: g.addV('person').property('name', 'John').next())
    add_statement(lambda: g.addV('person').property('name', 'Frank').next())
    add_statement(lambda: g.addV('person').property('name', 'Sally').next())
    add_statement(lambda: g.addV('person').property('name', 'Rob').property('marked', True).next())
    add_statement(lambda: g.addV('person').property('name', 'Bill').property('marked', True).next())
    add_statement(lambda: g.addV('person').property('name', 'Sal').property('marked', True).next())

with execute_gremlin_statements() as add_statement:
    # Add edges
    add_statement(lambda: g.V().has('name', 'Rob').addE('sent').to(__.V().has('name', 'Frank')).property('value', 8).next())
    add_statement(lambda: g.V().has('name', 'Rob').addE('sent').to(__.V().has('name', 'Sally')).property('value', 7).next())
    add_statement(lambda: g.V().has('name', 'Bill').addE('sent').to(__.V().has('name', 'Frank')).property('value', 3).next())
    add_statement(lambda: g.V().has('name', 'Bill').addE('sent').to(__.V().has('name', 'Sally')).property('value', 10).next())
    add_statement(lambda: g.V().has('name', 'Frank').addE('sent').to(__.V().has('name', 'Bob')).property('value', 5).next())
    add_statement(lambda: g.V().has('name', 'Frank').addE('sent').to(__.V().has('name', 'John')).property('value', 2).next())
    add_statement(lambda: g.V().has('name', 'Sally').addE('sent').to(__.V().has('name', 'Bob')).property('value', 3).next())
    add_statement(lambda: g.V().has('name', 'Sally').addE('sent').to(__.V().has('name', 'John')).property('value', 3).next())
    add_statement(lambda: g.V().has('name', 'Sal').addE('sent').to(__.V().has('name', 'Rob')).property('value', 17).next())


v[36992]
v[28680]
v[32776]
v[28880]
v[20576]
v[32872]
v[41088]


KeyError: <DataType.custom: 0>

### Find Coin Sources

In [ ]:
def get_sources(g, node_name):
    source_record = {}
    def traverse_sources(vertex, fraction=1.0):
        # Traverse incoming edges
        in_edges = g.V(vertex).inE('sent').toList()
        for edge in in_edges:
            sender = edge.outV().next()
            amount_from_sender = edge['value']
            if 'marked' in sender.properties:
                # Add record if sender is marked
                if sender['name'] in source_record:
                    source_record[sender['name']] += amount_from_sender * fraction
                else:
                    source_record[sender['name']] = amount_from_sender * fraction
            # Recursive case
            if g.V(sender).inE('sent').hasNext():
                sender_total_received = sum(e['value'] for e in g.V(sender).inE('sent').toList())
                amount_fraction = (amount_from_sender / sender_total_received) * fraction
                traverse_sources(sender, amount_fraction)

    # Start traversal
    traverse_sources(g.V().has('name', node_name).next())
    return source_record

print("Sources for Bob:")
print(get_sources(g, 'Bob'))

### Execute Some Queries

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# def add_vertex():
#     return g.addV('Address').property('addr', 'some_address').next()

# with ThreadPoolExecutor() as executor:
#     future = executor.submit(add_vertex)
#     result = future.result()
    # print(result)
    
with execute_gremlin_statements() as add_statement:
    add_statement(lambda: g.addV('Address').property('addr', 'some_address').next())

v[4288]
